# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file, encoding="UTF-8")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,21.42,60,86,5.26,MG,1693303899
1,1,belmonte,-15.8631,-38.8828,22.26,94,99,3.20,BR,1693303899
2,2,grytviken,-54.2811,-36.5092,1.18,91,99,5.06,GS,1693303900
3,3,keflavik,64.0049,-22.5624,11.95,82,40,4.12,IS,1693303901
4,4,la sarre,48.8002,-79.1996,12.41,91,100,1.78,CA,1693303901


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

coordinates = {
    "latitude": city_data_df['Lat'],
    "longitude": city_data_df['Lng'],
    "City": city_data_df['City'],
    "Humidity": city_data_df['Humidity']
}

coordinates_df = pd.DataFrame(coordinates)

#Configure the map plot
city_map_humidity = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM", 
    frame_width = 800,
    frame_height = 500, 
    size = "Humidity", 
    scale = 0.7,
    alpha = 0.7,
    color = "City",
    
)

# Display the map plot
city_map_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# My ideal temp Max temp <25 & Humidity <40%
my_weather_df = city_data_df.loc[(city_data_df['Max Temp'] < 25) & (city_data_df['Humidity'] < 40)].dropna()

# Display sample data
my_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,undurkhaan,47.3194,110.6556,21.97,39,73,2.48,MN,1693303905
26,26,astrakhan,46.3497,48.0408,21.93,28,0,7.00,RU,1693303912
68,68,tilcara,-23.5776,-65.3509,3.96,25,100,4.33,AR,1693303935
72,72,shalqar,47.8333,59.6000,19.44,26,17,7.70,KZ,1693303937
112,112,queenstown,-31.8976,26.8753,24.67,11,0,5.59,ZA,1693303957


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_weather_df[['City_ID','City','Country','Lat','Lng','Humidity']].set_index("City_ID", drop=True)
hotel_df.index.rename(None, inplace=True)

# # Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# # Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,undurkhaan,MN,47.3194,110.6556,39,
26,astrakhan,RU,46.3497,48.0408,28,
68,tilcara,AR,-23.5776,-65.3509,25,
72,shalqar,KZ,47.8333,59.6000,26,
112,queenstown,ZA,-31.8976,26.8753,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = (f"circle:{longitude},{latitude},{radius}")
    params["bias"] = (f"proximity:{longitude},{latitude}")
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

print ("-----------------------------------------------------------------------------------------")
# Display sample data
hotel_df

Starting hotel search
undurkhaan - nearest hotel: Khanburged Hotel
astrakhan - nearest hotel: Omega
tilcara - nearest hotel: Casa Colorada - Hotel de Montaña
shalqar - nearest hotel: Бекнур
queenstown - nearest hotel: Queens Hotel
rakops - nearest hotel: Rakops River Lodge
klyuchi - nearest hotel: No hotel found
altai - nearest hotel: Altai hotel 40000-100000T
chefchaouene - nearest hotel: Hotel Andalouz
qulsary - nearest hotel: Камилла
sao vicente - nearest hotel: Pousada Vitória
port augusta - nearest hotel: Hotel Commonwealth
alice springs - nearest hotel: Aurora Alice Springs
krasnyy kut - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
fiambala - nearest hotel: Cabañas Lunita
qarauyl - nearest hotel: No hotel found
at-bashi - nearest hotel: Tash Rabat
tupiza - nearest hotel: Aramayo Hostal
butha-buthe - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
grahamstown - nearest hotel: Colonel Graham Guest House
ghanzi - nearest hotel: Ghanzi f

,City,Country,Lat,Lng,Humidity,Hotel Name
12,undurkhaan,MN,47.3194,110.6556,39,Khanburged Hotel
26,astrakhan,RU,46.3497,48.0408,28,Omega
68,tilcara,AR,-23.5776,-65.3509,25,Casa Colorada - Hotel de Montaña
72,shalqar,KZ,47.8333,59.6000,26,Бекнур
112,queenstown,ZA,-31.8976,26.8753,11,Queens Hotel
114,rakops,BW,-21.0167,24.3333,28,Rakops River Lodge
118,klyuchi,RU,52.2667,79.1667,30,No hotel found
130,altai,MN,46.3722,96.2583,21,Altai hotel 40000-100000T
142,chefchaouene,MA,35.1688,-5.2636,16,Hotel Andalouz
159,qulsary,KZ,46.9531,54.0198,35,Камилла


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map plot
hotel_df_map 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)